# Ideia
- Carregar o Dataset
- Tratar as strings para gerar embeddings de maior qualidade
- Gerar os embeddings para a descrição dos produtos usando um modelo de linguagem
- calcular a similaridade do produto destacado com base na distância dos cossenos dos embeddings gerados (Vector Serach)

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Checking Data

In [12]:
df = pd.read_csv("/home/gui/Área de Trabalho/DS_challenge/DS_challenge/SourceFiles/items_titles.csv")
print(df.size)
df.head()

30000


,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...


In [15]:
df = pd.read_csv("/home/gui/Área de Trabalho/DS_challenge/DS_challenge/SourceFiles/items_titles_test.csv")
print(df.size)
df.head()

10000


,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde


# Utils

In [16]:
def generate_product_embeddings(texts, model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'):
    try:
        model = SentenceTransformer(model_name)
        embeddings = model.encode(texts, convert_to_numpy=True)
        return embeddings
    except Exception as e:
        print(f"Erro ao gerar embeddings: {e}")
        return None


In [17]:
def calculate_similarity_score(embedding1, embedding2):
    score = cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))[0][0]
    return score

## TEST

In [32]:
df_test = df.head(2).copy()
df_test['embeddings'] = generate_product_embeddings(df_test['ITE_ITEM_TITLE'].tolist()).tolist()

similarity_score = (calculate_similarity_score(np.array(df_test['embeddings'].iloc[0]), np.array(df_test['embeddings'].iloc[1])) + 1)/2

print(f"\nScore de Similaridade (Cosseno): {similarity_score:.4f}")
df_test


Score de Similaridade (Cosseno): 0.7299


,ITE_ITEM_TITLE,embeddings
0,Tênis Olympikus Esporte Valente - Masculino Kids,"[0.12636679410934448, 0.29663726687431335, -0...."
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,"[-0.2898196280002594, -0.0019043326610699296, ..."


# Embeddings

In [63]:
product_descriptions = df['ITE_ITEM_TITLE'].tolist()
product_embeddings = generate_product_embeddings(product_descriptions)
df["embeddings"] = generate_product_embeddings(product_descriptions).tolist()

df.head()

,ITE_ITEM_TITLE,emeddings,embeddings
0,Tênis Olympikus Esporte Valente - Masculino Kids,"[0.1263667643070221, 0.29663726687431335, -0.0...","[0.1263667643070221, 0.29663726687431335, -0.0..."
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,"[-0.2898196280002594, -0.0019043318461626768, ...","[-0.2898196280002594, -0.0019043318461626768, ..."
2,Tênis Usthemp Slip-on Temático - Labrador 2,"[0.039872851222753525, -0.0576791912317276, 0....","[0.039872851222753525, -0.0576791912317276, 0...."
3,Tênis Casual Feminino Moleca Tecido Tie Dye,"[0.018457120284438133, 0.16783775389194489, 0....","[0.018457120284438133, 0.16783775389194489, 0...."
4,Tênis Star Baby Sapatinho Conforto + Brinde,"[0.015894582495093346, 0.29608991742134094, -0...","[0.015894582495093346, 0.29608991742134094, -0..."


In [64]:
df.to_csv('/home/gui/Área de Trabalho/DS_challenge/DS_challenge/SourceFiles/items_titles_test_embeddings.csv')

# Similarity

In [ ]:
item = df.iloc[0]

def score(i,x):
    return (calculate_similarity_score(np.array(i.embeddings), np.array(x.embeddings)) + 1)/2

df[['REFF_ITEM_TITLE', 'score']] = df.apply(
    lambda x: (item.ITE_ITEM_TITLE, score(item, x)), 
    axis=1, 
    result_type="expand"
) 

df[['REFF_ITEM_TITLE', 'ITE_ITEM_TITLE', 'score']].head()

,REFF_ITEM_TITLE,ITE_ITEM_TITLE,score
0,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Olympikus Esporte Valente - Masculino Kids,1.000000
1,Tênis Olympikus Esporte Valente - Masculino Kids,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,0.729946
2,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Usthemp Slip-on Temático - Labrador 2,0.680565
3,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Casual Feminino Moleca Tecido Tie Dye,0.707611
4,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Star Baby Sapatinho Conforto + Brinde,0.805908
...,...,...,...
9995,Tênis Olympikus Esporte Valente - Masculino Kids,Chuteira Futsal Oxn Velox 3 Infantil,0.823688
9996,Tênis Olympikus Esporte Valente - Masculino Kids,Sapatenis Casual Masculino Estiloso 24horas Co...,0.771018
9997,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Feminino Infantil Molekinha Tie Dye,0.750203
9998,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Feminino Leve Barato Ganhe 1 Colchonete ...,0.798749


In [70]:
df[['REFF_ITEM_TITLE', 'ITE_ITEM_TITLE', 'score']].sort_values(by='score', ascending=False)

,REFF_ITEM_TITLE,ITE_ITEM_TITLE,score
0,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Olympikus Esporte Valente - Masculino Kids,1.000000
1767,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Infantil Masculino Olympikus 943 Supremo...,0.935747
7922,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Olympikus Masculino Valente - Kids Infan...,0.921823
792,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Infantil Masculino Olympikus Valente Azu...,0.906437
4381,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Infantil Olympikus Enjoy Kids Menino Con...,0.905943
...,...,...,...
2202,Tênis Olympikus Esporte Valente - Masculino Kids,Tv Samsung Led 32 Polegadas,0.472590
4364,Tênis Olympikus Esporte Valente - Masculino Kids,Smart Tv Tcl 40s6500 Led Full Hd 40 100v/240v,0.470137
4955,Tênis Olympikus Esporte Valente - Masculino Kids,Smart Tv Samsung 50 Polegadas Com Defeito Fun...,0.469385
8762,Tênis Olympikus Esporte Valente - Masculino Kids,Tv Semp Toshiba 21 Polegadas Tela Plana Cinza ...,0.464878
